# Breaking the check code

## Strange book:

>Recent advances in interdimensional physics have produced fascinating
>predictions about the fundamentals of our universe!  For example,
>interdimensional physics seems to predict that the universe is, at its root, a
>purely mathematical construct, and that all events are caused by the
>interactions between eight pockets of energy called "registers".
>Furthermore, it seems that while the lower registers primarily control mundane
>things like sound and light, the highest register (the so-called "eighth
>register") is used to control interdimensional events such as teleportation.
>
>A hypothetical such teleportation device would need to have have exactly two
>destinations.  One destination would be used when the eighth register is at its
>minimum energy level - this would be the default operation assuming the user
>has no way to control the eighth register.  In this situation, the teleporter
>should send the user to a preconfigured safe location as a default.
>
>The second destination, however, is predicted to require a very specific
>energy level in the eighth register.  The teleporter must take great care to
>confirm that this energy level is exactly correct before teleporting its user!
>If it is even slightly off, the user would (probably) arrive at the correct
>location, but would briefly experience anomalies in the fabric of reality
>itself - this is, of course, not recommended.  Any teleporter would need to test
>the energy level in the eighth register and abort teleportation if it is not
>exactly correct.
>
>This required precision implies that the confirmation mechanism would be very
>computationally expensive.  While this would likely not be an issue for large-
>scale teleporters, a hypothetical hand-held teleporter would take billions of
>years to compute the result and confirm that the eighth register is correct.
>
>If you find yourself trapped in an alternate dimension with nothing but a
>hand-held teleporter, you will need to extract the confirmation algorithm,
>reimplement it on more powerful hardware, and optimize it.  This should, at the
>very least, allow you to determine the value of the eighth register which would
>have been accepted by the teleporter's confirmation mechanism.
>
>Then, set the eighth register to this value, activate the teleporter, and
>bypass the confirmation mechanism.  If the eighth register is set correctly, no
>anomalies should be experienced, but beware - if it is set incorrectly, the
>now-bypassed confirmation mechanism will not protect you!
>
>Of course, since teleportation is impossible, this is all totally ridiculous.

## Finding the check code

Start to seek every line where register7 is used. There are 4 of those. By debugging the machine we can determin which line is just testing:
* which if register7 is empty on begining
* test when telepert is used (if register7 is 0 normall behaviour)
* passing value of register7 to check function (actually inside of it)
* passing value of register7 to final function which calculating the secret code

## Original code of check function

```
*   6027:      jt  register_0   6035
    6030:     add  register_0   register_1   1
    6034:     ret

    6035:      jt  register_1   6048
    6038:     add  register_0   register_0   32767
    6042:     set  register_1   register_7
    6045:    call  6027
    6047:     ret

    6048:    push  register_0
    6050:     add  register_1   register_1   32767
    6054:    call  6027
    6056:     set  register_1   register_0
    6059:     pop  register_0
    6061:     add  register_0   register_0   32767
    6065:    call  6027
    6067:     ret
```

In [1]:
import pandas
pandas.set_option("precision", 0)

MAX_INT = 32768

def add32767(value):
    return (value + 32767) % MAX_INT

pandas.DataFrame(
    ({
        'x in function': f'({i} + 32767) % {MAX_INT}',
        'result': add32767(i),
        'approximation': f'({i} - 1) % {MAX_INT}'
    } for i in range(10))
)

,x in function,result,approximation
0,(0 + 32767) % 32768,32767,(0 - 1) % 32768
1,(1 + 32767) % 32768,0,(1 - 1) % 32768
2,(2 + 32767) % 32768,1,(2 - 1) % 32768
3,(3 + 32767) % 32768,2,(3 - 1) % 32768
4,(4 + 32767) % 32768,3,(4 - 1) % 32768
5,(5 + 32767) % 32768,4,(5 - 1) % 32768
6,(6 + 32767) % 32768,5,(6 - 1) % 32768
7,(7 + 32767) % 32768,6,(7 - 1) % 32768
8,(8 + 32767) % 32768,7,(8 - 1) % 32768
9,(9 + 32767) % 32768,8,(9 - 1) % 32768


This function can be translated then to:

$$
\begin{array}{lcl}
f(0, y, n) & = & y + 1 \\
f(x, 0, n) & = & f(x - 1, n, n) \\
f(x, y, n) & = & f(x - 1, f(x, y - 1, n), n)
\end{array}
$$

In Python:

In [2]:
from functools import lru_cache

@lru_cache(maxsize=None)
def f(x, y, n):
    if x == 0:
        return (y + 1) % MAX_INT

    if y == 0:
        return f(x - 1, n, n)

    return f(x - 1, f(x, y - 1, n), n)

Unfortunately this function cannot be directly express as single calculation, or replace by loop.
Let's try something generate few values in order to find some patterns for optimalizations.

In [3]:
def test(x, y, reg7, expected, result):
    assert expected == result, f'Error for {x}, {y}, {reg7}, excepted {expected}, was: {result}'


paramaters = [(x, y) for x in range(1, 3) for y in range(5)] + [(3, y) for y in range(4)]
test_results = [(x, y, n, f(x, y, n)) for x, y in paramaters for n in range(10) if not (x == 3 and y == 3 and n > 5)]

out = {}
for test_result in test_results:
    x, y, n, result = test_result
    label = f'f({x}, {y}, n)'
    n_result = out.get(label, {})
    n_result[n] = result
    out[label] = n_result

pandas.DataFrame(out).transpose()

,0,1,2,3,4,5,6,7,8,9
"f(1, 0, n)",1,2,3,4,5,6,7,8,9,10
"f(1, 1, n)",2,3,4,5,6,7,8,9,10,11
"f(1, 2, n)",3,4,5,6,7,8,9,10,11,12
"f(1, 3, n)",4,5,6,7,8,9,10,11,12,13
"f(1, 4, n)",5,6,7,8,9,10,11,12,13,14
"f(2, 0, n)",1,3,5,7,9,11,13,15,17,19
"f(2, 1, n)",2,5,8,11,14,17,20,23,26,29
"f(2, 2, n)",3,7,11,15,19,23,27,31,35,39
"f(2, 3, n)",4,9,14,19,24,29,34,39,44,49
"f(2, 4, n)",5,11,17,23,29,35,41,47,53,59


It is easy to calculate the $ f(1, y, n) $. Let us expand it:

$$
\begin{array}{lcl}
f(1, y, n) & = & f(0, f(1, y - 1, n), n) \\
           & = & 1 + f(1, y - 1, n) \\
           & = & 1 + 1 + f(1, y - 2, n) \\
           & = &  ... \\
           & = & \sum^{y} 1 + f(1, 0, n) \\
           & = & \sum^{y} 1 + f(0, n, n) \\
           & = & \sum^{y} 1 + n + 1 \\
           & = & y + n + 1
\end{array}
$$


In [4]:
for x, y, n, expected in test_results:
    if x == 1:
        test(x, y, n, expected, n + 1 + y)

print('f(1, y) -> n + 1 + y')

f(1, y) -> n + 1 + y


In [5]:
@lru_cache(maxsize=None)
def f_v1(x, y, n):
    if x == 0:
        return (y + 1) % MAX_INT
    if x == 1:
        return (n + y + 1) % MAX_INT

    if y == 0:
        return f_v1(x - 1, n, n)

    return f_v1(x - 1, f_v1(x, y - 1, n), n)

for x, y, n, expected in test_results:
    test(x, y, n, expected, f_v1(x, y, n))

print('f_v1 is equivalent to f')

f_v1 is equivalent to f


In [6]:
for x, y, n, expected in test_results:
    if x == 2:
        test(x, y, n, expected, (x + y) * n + (x + y - 1))

print('f(2, y) -> (x + y) * n + (x + y - 1) ')

f(2, y) -> (x + y) * n + (x + y - 1) 


In [7]:
@lru_cache(maxsize=None)
def f_v2(x, y, n):
    if x == 0:
        return (y + 1) % MAX_INT
    if x == 1:
        return (n + 1 + y) % MAX_INT
    if x == 2:
        return ((x + y) * n + (x + y - 1)) % MAX_INT

    if y == 0:
        return f_v2(x - 1, n, n)
   
    return f_v2(x - 1, f_v2(x, y - 1, n), n)

for x, y, n, expected in test_results:
    test(x, y, n, expected, f_v2(x, y, n))

print('f_v2 is equivalent to f')


f_v2 is equivalent to f


In [8]:
approximations = {
    # 1 + n (3 + n)
    (3, 0): lambda n: (n + 1) ** 2 + n,                                  # n^2 + 3 n + 1

    # 2 + n (6 + n (4 + n))
    (3, 1): lambda n: (n + 1) ** 2 * (n + 2) + n,                        # n^3 + 4 n^2 + 6 n + 2

    # 3 + n (10 + n (10 + n (5 + n)))
    (3, 2): lambda n: (n + 1) ** 3 * (n + 2) + (n + 1)**2 + n,           # n^4 + 5 n^3 + 10 n^2 + 10 n + 3

    # 4 + n (15 + n (20 + n (15 + n (6 + n))))
    (3, 3): lambda n: (n + 1) ** 4 * (n + 2) + (n + 1)**2 * (n + 2) + n  # n^5 + 6 n^4 + 15 n^3 + 20 n^2 + 15 n + 4
}

keys = list(range(len(approximations)))
pandas.DataFrame(
    [[approximations[(3, y)](n) for n in range(10)] for y in keys],
    (f'f(3, {y}, n)' for y in keys))

,0,1,2,3,4,5,6,7,8,9
"f(3, 0, n)",1,5,11,19,29,41,55,71,89,109
"f(3, 1, n)",2,13,38,83,154,257,398,583,818,1109
"f(3, 2, n)",3,29,119,339,779,1553,2799,4679,7379,11109
"f(3, 3, n)",4,61,362,1363,3904,9329,19606,37447,66428,111109


If you look on the multiplication for next n element in the polynomial, the pattern emerge:

| Function | Polynomial |N multiplaries|
|----------|------------|--------------|
| `f(3, 0, n)` | n^2 + 3 n + 1 | 1, 3 |
| `f(3, 1, n)` | n^3 + 4 n^2 + 6 n + 2 | 1, 4, 6 |
| `f(3, 2, n)` | n^4 + 5 n^3 + 10 n^2 + 10 n + 3| 1, 5, 10, 10 |
| `f(3, 3, n)` | n^5 + 6 n^4 + 15 n^3 + 20 n^2 + 15 n + 4 | 1, 6, 15, 20, 15 |


In [9]:
def pascal_triangle():
    layer = [1]

    while True:
        yield layer
        layer = [1] + list(x + y for x, y in zip(layer, layer[1:])) + [1]


g = pascal_triangle()
for row in range(13):
    print(f'{row:2}:', '{:^80}'.format(' '.join(map('{:^5}'.format, next(g)))))

 0:                                        1                                        
 1:                                     1     1                                     
 2:                                  1     2     1                                  
 3:                               1     3     3     1                               
 4:                            1     4     6     4     1                            
 5:                         1     5    10    10     5     1                         
 6:                      1     6    15    20    15     6     1                      
 7:                   1     7    21    35    35    21     7     1                   
 8:                1     8    28    56    70    56    28     8     1                
 9:             1     9    36    84    126   126   84    36     9     1             
10:          1    10    45    120   210   252   210   120   45    10     1          
11:       1    11    55    165   330   462   462   330   165   55

Triangle is a bit trimmed (by two last digits in each row).

Therefore the general equation for f(3, y, n) is: $$ f(3, y, n) = \sum_{k=0}^{y+1} {{y + 3} \choose k} n^{y + 2 - k} + y + 1 $$

In [10]:
from operator import mul
from functools import reduce

@lru_cache(maxsize=None)
def binomial_coefficient(n: int, k: int) -> int:
    k = min(k, n - k)
    if n == 0 or k == 0: return 1

    return reduce(mul, range(n, n - k, -1)) // reduce(mul, range(1, k + 1))


assert binomial_coefficient(1, 1) == 1
assert binomial_coefficient(6, 2) == 15
assert binomial_coefficient(12, 9) == 220
assert binomial_coefficient(10, 7) == 120

In [11]:
for x, y, n, expected in test_results:
    if x == 3:
        test(x, y, n, expected, sum(binomial_coefficient(y + 3, k) * n ** (y + 2 - k) for k in range(y + 2)) + y + 1)

print('f(3, y, n) -> sum(binomial_coefficient(y + 3, k) * n ** (y + 2 - k) for k in range(y + 2)) + y + 1')

f(3, y, n) -> sum(binomial_coefficient(y + 3, k) * n ** (y + 2 - k) for k in range(y + 2)) + y + 1


In [12]:
@lru_cache(maxsize=None)
def f_v3(x, y, n):
    if x == 0:
        return (y + 1) % MAX_INT
    if x == 1:
        return (n + 1 + y) % MAX_INT
    if x == 2:
        return ((x + y) * n + x + y - 1) % MAX_INT
    if x == 3:
        return (sum(binomial_coefficient(y + 3, k) * n ** (y + 2 - k) for k in range(y + 2)) + y + 1) % MAX_INT

    if y == 0:
        return f_v3(x - 1, n, n)
   
    return f_v3(x - 1, f_v3(x, y - 1, n), n)

for x, y, n, expected in test_results:
    test(x, y, n, expected, f_v3(x, y, n))

print('f_v3 is equivalent to f')

f_v3 is equivalent to f


It is worth to point out expression of $ f(4, 1, n) $:

$$
\begin{array}{lcl}
f(4, 1, n) & = & f(3, f(4, 0, n), n) \\
           & = & f(3, f(3, n, n), n)
\end{array}
$$

The use of binomial_coefficient is not very effective. There for it require change of appreach. Going back to $ f(2, y, n) $ form:

$$
\begin{array}{lcl}
f(2, y, n) & = & (x + y) n + x + y - 1 \\
           & = & xn + yn + x + y - 1 \\
           & = & x(n + 1) + y(n + 1) - 1 \\
           & = & (x + y)(n + 1) - 1
\end{array}
$$

Then transform $ f(3, y, n) $

$$
\begin{array}{lcl}
f(3, y, n) & = & f(2, f(3, y - 1, n), n) \\
           & = & (2 + f(3, y - 1, n))(n + 1) - 1
\end{array}
$$

In this form is closer to dynamic programic form.


In [13]:
%%time

def calculate_f_4_1(n: int) -> int:
    cache = { 0: approximations[(3, 0)](n) }
    for i in range(1, MAX_INT):
        cache[i] = ((2 + cache[i - 1]) * (n + 1) - 1) % MAX_INT

    return cache[cache[n]]

for n in range(1, MAX_INT):
    if calculate_f_4_1(n) == 6:
        print('The result:', n)

The result: 25734
CPU times: user 4min 57s, sys: 16.9 s, total: 5min 14s
Wall time: 5min 14s
